In [ ]:
import site
site.addsitedir(os.path.curdir + '\..')
from analysis import *
import scipy as sp
# ^ this has all of the analysis functions from the prior notebooks

In [ ]:
# hyperparameter_path = "C:\\Users\\mickey\\aoLab\\code\\hierarchical_lfads\\hyperparameters\\ecog\\lfads_ecog_4.yaml"
# data_path = "D:\\Users\\mickey\\Data\\datasets\\ecog\\goose_wireless\\gw_250_fl0u10"
data_path = "D:\\Users\\mickey\\Data\\datasets\\ecog\\goose_wireless\\gw_250"
overwrite = False
# assert os.path.exists(hyperparameter_path), "hyperparameter file not found."
assert os.path.exists(data_path), "data file not found."
# model_dir = "D:\\Users\\mickey\\Data\\models\\pyt\\lfads\\gw_250_fl0u10\\lfads_ecog\\"
model_dir = "D:\\Users\\mickey\\Data\\models\\pyt\\lfads\\gw_250\\lfads_ecog\\"
# model_dir_list = glob(os.path.join(model_dir,"cenc0_cont0_fact128_genc256_gene256_glat256_nch*_seqlen*_ulat0_orion-l1"))
# model_dir_list = glob(os.path.join(model_dir,"cenc0_cont0_fact64_genc256_gene256_glat256_nch*_seqlen*_ulat0_orion-"))
# model_dir_list = glob(os.path.join(model_dir,"cenc0_cont0_fact128_genc1024_gene1024_glat1024_nch*_seqlen*_ulat0_orion-"))
model_dir_list = glob(os.path.join(model_dir,"cenc0_cont0_fact64_genc1024_gene1024_glat1024_nch42_seqlen50_ulat0_orion-varstd"))
# model_dir_list = list(set(model_dir_list) | set(glob(os.path.join(model_dir,"cenc0_cont0_fact128_genc1024_gene1024_glat1024_nch*_seqlen*_ulat0_orion-"))))
metric_stat_table_file = os.path.join(model_dir,"stat_table_freq.csv")
if os.path.exists(metric_stat_table_file) and not overwrite:
    metric_stat_table = pd.read_csv(metric_stat_table_file)
    run_model_dir_list = list(metric_stat_table['model_path'])
    model_dir_list = list(set(model_dir_list) - set(run_model_dir_list))
    metric_stat_table = [pd.read_csv(metric_stat_table_file)]
else:
    metric_stat_table = []
print(f"unassessed models found:\t{len(model_dir_list)}")
print(model_dir_list)

In [ ]:
n_trace = 6
srate = 250
n_boot = 500
metric_list = []
ar_model_path = 'D:\\Users\\mickey\\Data\\analysis\\ar_valid_set\\ar_model_dict.pkl'
model_perf_table = 'D:\\Users\\mickey\\Data\\analysis\\freq-band-test\\stat_table.csv'
with open(ar_model_path,'rb') as f:
    ar_model_dict = pkl.load(f)
for model_dir in tqdm(model_dir_list[:1]):
    hyperparameter_path = os.path.join(model_dir,'hyperparameters.yaml')
    print(f'loading model from:\t{model_dir}')
    model, test_data, test_data_mask = load_configure_model_data(model_dir,data_path,hyperparameter_path)
    b_lp, a_lp = sp.signal.iirfilter(10,40./(250/2),btype='lowpass')
    # b_hp, a_hp = sp.signal.iirfilter(10,40./(250/2),btype='highpass')
    test_data = np.float32(sp.signal.filtfilt(b_lp, a_lp, test_data, axis=1))
    # test_data = np.float32(sp.signal.filtfilt(b_hp, a_hp, test_data, axis=1))
    test_data = test_data[np.any(test_data.std(axis=1)>0.5,axis=-1),:,:]
    # test_data = sp.stats.zscore(test_data,axis=1)
    print('computing test data reconstructions...')
    recon, _, _ = compute_model_outputs(model,torch.tensor(test_data))
    print('computing metric statistics...')
    stat_table, metric_dict = compute_metric_table(test_data,recon,model_dir)
# if os.path.exists(model_perf_table):
#     stat_table.to_csv(model_perf_table,mode='a',header=False)
# else:
#     stat_table.to_csv(model_perf_table,mode='w',header=True)

In [ ]:
err = test_data - np.array(recon['data'])
# err_lf = test_data_lf - np.array(recon_lf['data'])
# err_hf = test_data_hf - np.array(recon_hf['data'])
# test_data.shape
test_hat = np.fft.rfft(test_data,axis=1)
# test_lf_hat = np.fft.rfft(test_data_hf,axis=1)
# test_hf_hat = np.fft.rfft(test_data_hf,axis=1)
recon_hat = np.fft.rfft(np.array(recon['data']),axis=1)
# recon_lf_hat = np.fft.rfft(np.array(recon_lf['data']),axis=1)
# recon_hf_hat = np.fft.rfft(np.array(recon_hf['data']),axis=1)
err_hat = np.fft.rfft(err,axis=1)
# err_lf_hat = np.fft.rfft(err_lf,axis=1)
# err_hf_hat = np.fft.rfft(err_hf,axis=1)
freq = np.fft.rfftfreq(50,d=1/srate)
time = np.arange(50)/srate

In [ ]:
trial_idx = np.random.randint(0,high=test_data.shape[0])
fig, ax = plt.subplots(3,1,dpi=150,constrained_layout=True)
ax = ax.reshape(-1)
ax[0].plot(time,test_data[trial_idx,:,0],label='data')
ax[0].plot(time,recon['data'][trial_idx,:,0],label='recon.')
ax[0].plot(time,err[trial_idx,:,0],label='err')
ax[0].set_xlabel('time (s)')
ax[0].legend(loc=0)
ax[1].plot(freq,20*np.log10(np.abs(test_hat[trial_idx,:,0])))
ax[1].plot(freq,20*np.log10(np.abs(recon_hat[trial_idx,:,0])))
ax[1].plot(freq,20*np.log10(np.abs(err_hat[trial_idx,:,0])))
ax[1].set_xlabel('freq. (Hz)')
# ax[2].plot(freq,np.angle(test_hat[trial_idx,:,0]))
# ax[2].plot(freq,np.angle(recon_hat[trial_idx,:,0]))
# ax[2].plot(freq,np.angle(err_hat[trial_idx,:,0]))
ax[2].plot(freq,20*(np.log10(np.abs(err_hat[trial_idx,:,0]))-np.log10(np.abs(test_hat[trial_idx,:,0]))))
ax[2].set_title('Normalized Error')

In [ ]:
# that's a good plot: let's look at the distribution of all these normalized error values
f_est = lambda x: np.nanmean(x,axis=(0,2))
# data
test_hat_bsd = bootstrap_est(test_hat,n_boot,f_est)
test_hat_mean = np.nanmean(test_hat_bsd,axis=0)
test_hat_ci = np.nanpercentile(test_hat_bsd,[2.5,97.5],axis=0)
# reconstruction
recon_hat_bsd = bootstrap_est(recon_hat,n_boot,f_est)
# normalized error
norm_err_hat = np.log10(np.abs(err_hat)) - np.log10(np.abs(test_hat))
norm_err_hat[np.isposinf(norm_err_hat)] = np.nan
norm_err_hat_bsd = bootstrap_est(norm_err_hat,n_boot,f_est)
norm_err_hat_mean = np.nanmean(norm_err_hat_bsd,axis=0)
norm_err_hat_ci = np.nanpercentile(norm_err_hat_bsd,[2.5,97.5],axis=0)

In [ ]:
plt.fill_between(freq,20*norm_err_hat_ci[0,:],20*norm_err_hat_ci[1,:],alpha=0.3)
plt.plot(freq,20*norm_err_hat_mean)

In [ ]:
b_lp, a_lp = sp.signal.iirfilter(10,40./(250/2),btype='lowpass')
b_hp, a_hp = sp.signal.iirfilter(10,40./(250/2),btype='highpass')
test_data_lp = sp.signal.filtfilt(b_lp, a_lp, test_data, axis=1)
test_data_hp = sp.signal.filtfilt(b_hp, a_hp, test_data, axis=1)

In [ ]:
plt.plot(time,test_data[0,:,14])
plt.plot(time,test_data_lp[0,:,14])
plt.plot(time,test_data_hp[0,:,14])